This file is to get similarity among users. We set the user 'P3029' as 'Me' and the other as roommates. The similarity we were trying to obtain is calculated by comparing users' sleep time.

In [115]:
import pandas as pd
import os
from datetime import datetime, timedelta
import numpy as np

PATH = os.getcwd()
PATH = "../csv/routines_raw/sleep/"
file_list = os.listdir(PATH)

In [116]:
user_data = pd.read_csv(PATH + "P3029-sleep.csv")
roommate_data = dict()
for file in file_list:
    if file == "P3029-sleep.csv":
        continue
    roommate_data[file.split('.')[0]] = pd.read_csv(PATH + file)

In [117]:
def get_similarity(roommate, user=user_data):
    user_start_at = pd.to_datetime(user['start_at'])
    user_end_at = pd.to_datetime(user['end_at'])
    roommate_start_at = pd.to_datetime(roommate['start_at'])
    roommate_end_at = pd.to_datetime(roommate['end_at'])
    
    time = pd.concat([user_start_at, user_end_at, roommate_start_at, roommate_end_at], axis=1)
    time.dropna(axis=0)
    time.columns = ['user_start_at', 'user_end_at', 'roommate_start_at', 'roommate_end_at']

    time['union_start_at'] = time.apply(lambda x: x.user_start_at if x.user_start_at < x.roommate_start_at else x.roommate_start_at, axis=1)
    time['union_end_at'] = time.apply(lambda x: x.user_end_at if x.user_end_at > x.roommate_end_at else x.roommate_end_at, axis=1)
    time['intersect_start_at'] = time.apply(lambda x: x.user_start_at if x.user_start_at > x.roommate_start_at else x.roommate_start_at, axis=1)
    time['intersect_end_at'] = time.apply(lambda x: x.user_end_at if x.user_end_at < x.roommate_end_at else x.roommate_end_at, axis=1)
    
    # print(f"intersect: {(time['intersect_end_at'] - time['intersect_start_at']).sum()}")
    # print(f"union: {(time['union_end_at'] - time['union_start_at']).sum()}")
    similarity = abs(round((time['intersect_end_at'] - time['intersect_start_at']).sum()/(time['union_end_at'] - time['union_start_at']).sum(), 2))
    # print(f"similarity = {similarity}")
    return similarity

In [118]:
get_similarity(user_data, roommate_data["P3030-sleep"])

0.54

In [119]:
similarity_df = pd.DataFrame(columns={'user_id', 'similarity'})
for (file_name, df) in roommate_data.items():
    similarity = get_similarity(df)
    similarity_df = similarity_df.append({'user_id': file_name.split('-')[0], 'similarity': similarity}, ignore_index=True)
similarity_df

,user_id,similarity
0,P3018,0.68
1,P3019,0.29
2,P3021,0.79
3,P3022,0.51
4,P3023,0.82
5,P3024,0.34
6,P3025,0.46
7,P3027,0.72
8,P3028,0.70
9,P3030,0.54


In [120]:
similarity_df.to_csv(f'../csv/similarity.csv', index=False)